**Some vehicles stopped in violation are exempt from fines due to business reasons. For vehicles that are exempt, are there repeat offenders? Where are exempt vehicles frequently in violation?**

*Recommendation - This problem is great for a mapping visualization! Choose a CUNY bus route that you are familiar with plot the long and latitude of where vehicles are in violation as well as the overall bus route.*

## Data Extraction via API

In [8]:
import requests
import random
import logging
import time
import csv
import pandas as pd
import os
import io

DATA_DIR_RAW = os.path.join("..", "..", "data", "raw")
DATA_DIR_PROCESSED = os.path.join("..", "..", "data", "processed")

# 🪵 Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [9]:
def write_to_csv(data, output_file, replace=False):
    if os.path.exists(output_file) and not replace:
        logging.info(f"☑️ File '{output_file}' already exists. Skipping write.")
        return

    try:
        if isinstance(data, pd.DataFrame):
            # Handle DataFrame directly
            if data.empty:
                logging.warning("⚠️ No data to write (DataFrame is empty).")
                return
            data.to_csv(output_file, index=False, encoding="utf-8")
            logging.info(f"📄 Successfully wrote {len(data)} rows (DataFrame) to '{output_file}'")

        elif isinstance(data, list):
            # Handle list of dicts
            if not data:
                logging.warning("⚠️ No data to write (list is empty).")
                return
            fieldnames = sorted(set().union(*(d.keys() for d in data)))
            with open(output_file, mode="w", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            logging.info(f"📄 Successfully wrote {len(data)} rows (list of dicts) to '{output_file}'")

        else:
            logging.error(f"❌ Unsupported data type: {type(data)}. Expected DataFrame or list of dicts.")

    except Exception as e:
        logging.error(f"❌ Failed to write CSV: {e}")

In [ ]:
ACE_endpoint_url = "https://data.ny.gov/resource/kh8p-hcbm.csv"
CBD_endpoint_url = "https://data.ny.gov/resource/cgzt-smqf.csv"

exempt_types = (
    "'EXEMPT - BUS/PARATRANSIT', "
    "'EXEMPT - OTHER', "
    "'EXEMPT - EMERGENCY VEHICLE', "
    "'EXEMPT - COMMERCIAL UNDER 20'"
)

count_query = f"""
    SELECT count(*)
    WHERE violation_status IN ({exempt_types})
"""
response = requests.get(ACE_endpoint_url, params={"$query": count_query})

# Fix: parse the count
count_df = pd.read_csv(io.StringIO(response.text))
max_rows = int(count_df.iloc[0, 0])
print(f"Total exempt rows available: {max_rows}")

all_data = []
limit = 50000
offset = 0

while offset < max_rows:
    print(f"Fetching rows {offset + 1} to {min(offset + limit, max_rows)}")

    soql_query = f"""
        SELECT *
        WHERE violation_status IN ({exempt_types})
        LIMIT {limit}
        OFFSET {offset}
    """

    response = requests.get(ACE_endpoint_url, params={"$query": soql_query})

    try:
        chunk_data = pd.read_csv(io.StringIO(response.text))
        if not chunk_data.empty:
            all_data.append(chunk_data)
    except pd.errors.EmptyDataError:
        break

    offset += limit
    time.sleep(1)

ACE_exempt_violations = pd.concat(all_data, ignore_index=True)
print(f"Total rows collected: {len(ACE_exempt_violations)}")

Total exempt rows available: 870810
Fetching rows 1 to 50000
Fetching rows 50001 to 100000
Fetching rows 100001 to 150000
Fetching rows 150001 to 200000
Fetching rows 200001 to 250000
Fetching rows 250001 to 300000
Fetching rows 300001 to 350000
Fetching rows 350001 to 400000
Fetching rows 400001 to 450000
Fetching rows 450001 to 500000
Fetching rows 500001 to 550000
Fetching rows 550001 to 600000
Fetching rows 600001 to 650000
Fetching rows 650001 to 700000
Fetching rows 700001 to 750000
Fetching rows 750001 to 800000
Fetching rows 800001 to 850000
Fetching rows 850001 to 870810
Total rows collected: 870810


In [12]:
OUTPUT_ACE = os.path.join(DATA_DIR_RAW, "ACE_exempt_violations.csv")
write_to_csv(ACE_exempt_violations, OUTPUT_ACE)

2025-09-21 19:00:04,590 - INFO - 📄 Successfully wrote 870810 rows (DataFrame) to '..\..\data\raw\ACE_exempt_violations.csv'
